In [31]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

import zipfile
import warnings
warnings.filterwarnings('ignore')

import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchcontrib.optim import SWA

from sklearn.metrics import recall_score
from sklearn.utils import shuffle
from sklearn.model_selection import RepeatedStratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import f1_score

import albumentations as A
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
import gc


import joblib
from tqdm.auto import tqdm
print('cuda on : ', torch.cuda.is_available())

cuda on :  True


In [32]:
name_label_dict = {
0:  'Nucleoplasm',
1:  'Nuclear membrane',
2:  'Nucleoli',   
3:  'Nucleoli fibrillar center',
4:  'Nuclear speckles',
5:  'Nuclear bodies',
6:  'Endoplasmic reticulum',   
7:  'Golgi apparatus',
8:  'Peroxisomes',
9:  'Endosomes',
10:  'Lysosomes',
11:  'Intermediate filaments',
12:  'Actin filaments',
13:  'Focal adhesion sites',   
14:  'Microtubules',
15:  'Microtubule ends',  
16:  'Cytokinetic bridge',   
17:  'Mitotic spindle',
18:  'Microtubule organizing center',  
19:  'Centrosome',
20:  'Lipid droplets',
21:  'Plasma membrane',   
22:  'Cell junctions', 
23:  'Mitochondria',
24:  'Aggresome',
25:  'Cytosol',
26:  'Cytoplasmic bodies',   
27:  'Rods & rings' }

In [33]:
data_dir = "/Users/asdlk/Downloads"

In [34]:
en1 = pd.read_csv(f'{data_dir}/Resn_4ch_TTA_42ep_TTAmax8.csv')

In [35]:
en2 = pd.read_csv(f'{data_dir}/Effv2_b3_4ch_Ran__20ep_TTAmax8.csv')

In [36]:
en3 = pd.read_csv(f'{data_dir}/Resnet34_TTA_bce_43ep_TTA(8).csv') #protein_classification_TTA.csv

In [37]:
reverse_train_labels = dict((v,k) for k,v in name_label_dict.items())

In [38]:
for key in name_label_dict.keys():
    en1[name_label_dict[key]] = 0
for key in name_label_dict.keys():
    en2[name_label_dict[key]] = 0
for key in name_label_dict.keys():
    en3[name_label_dict[key]] = 0

In [39]:
en1[en1.Predicted.isnull()] = '-1'
en2[en2.Predicted.isnull()] = '-1'
en3[en3.Predicted.isnull()] = '-1'

In [40]:
def fill_targets(row):
    row.Predicted = np.array(row.Predicted.split(" ")).astype(np.int)  # label -> list(int)로 변경 index matching
    for num in row.Predicted:
        try:
            name = name_label_dict[int(num)]
            row.loc[name] = 1
        except:
            pass
    return row

en1 = en1.apply(fill_targets, axis=1)   #label_dict와 Target match
en2 = en2.apply(fill_targets, axis=1) 
en3 = en3.apply(fill_targets, axis=1) 

In [41]:
e1 = en1.iloc[:,2:]

In [42]:
e2 = en2.iloc[:,2:]

In [43]:
e3 = en3.iloc[:,2:]

In [44]:
e1 = np.array(e1).astype('int')

In [45]:
e2 = np.array(e2).astype('int')

In [46]:
e3 = np.array(e3).astype('int')

In [47]:
ensemble = e1 + e2 + e3


In [48]:
ensemble = np.where(ensemble >= 3, 1, 0)

In [49]:
submit = pd.read_csv(f'/Users/asdlk/Desktop/workplace/Inputs/human-protein-atlas-image-classification/sample_submission.csv')

In [50]:
def save_pred(pred, th=0.5, fname= 'ensemble_vote_3'):
    pred_list = []
    for line in tqdm(pred):
        s = ' '.join(list([str(i) for i in np.nonzero(line >th)[0]]))
        pred_list.append(s)
    submit['Predicted'] = pred_list
    submit.to_csv(f'./{fname}.csv',  index=False)


In [51]:
save_pred(ensemble)

  0%|          | 0/11702 [00:00<?, ?it/s]

In [22]:

0 % 5

0

In [23]:
1 % 5 

1

In [24]:
5 % 5

0

In [25]:
0 // 5

0

In [26]:
1 //5

0

In [27]:
2 // 5

0

In [28]:
5 // 5

1

In [29]:
6 // 5

1

In [30]:
10 // 5

2